In [2]:
%pip install mlflow
%pip install dagshub
%pip install optuna



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler, PowerTransformer, OrdinalEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

import mlflow
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import optuna

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## We will drop the null values and then use the diff models


In [4]:
import dagshub
dagshub.init(repo_owner='mukeshjangid7877', repo_name='zomato-delivery-time-prediction', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=a27a1c72-69bc-47c4-92be-1f31fdbe6f73&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a8a602e42e7bd4341d80f4b1f2ccf79c426196c4e639da811cb683c3eec3c1ba




Accessing as mukeshjangid7877

Initialized MLflow to track repo "mukeshjangid7877/zomato-delivery-time-prediction"

Repository mukeshjangid7877/zomato-delivery-time-prediction initialized!

In [5]:
df = pd.read_csv('/content/cleaned_data.csv')

In [6]:
df.head()

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather_conditions,traffic,...,time_taken (min),city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance
0,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,2022-12-02,Fog,jam,...,46,DEH,2,12,friday,0,15.0,21.0,night,10.28
1,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,2022-02-13,Stormy,high,...,23,KOC,13,2,sunday,1,10.0,14.0,afternoon,6.24
2,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,2022-04-03,Sandstorms,medium,...,21,PUNE,3,4,sunday,1,10.0,17.0,evening,13.79
3,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,2022-02-13,Sandstorms,low,...,20,LUDH,13,2,sunday,1,10.0,9.0,morning,2.93
4,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,2022-02-14,Fog,jam,...,41,KNP,14,2,monday,0,15.0,19.0,evening,19.40


In [7]:
## Setting tracking server

mlflow.set_tracking_uri("https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow")


In [10]:
mlflow.set_experiment("Exp - 2 model selection")

2025/02/08 15:54:20 INFO mlflow.tracking.fluent: Experiment with name 'Exp - 2 model selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/26d3bc4e00974dbfb3c21d48f70798ba', creation_time=1739030060004, experiment_id='2', last_update_time=1739030060004, lifecycle_stage='active', name='Exp - 2 model selection', tags={}>

In [11]:
# drop columns not required for model input

columns_to_drop =  ['rider_id',
                    'restaurant_latitude',
                    'restaurant_longitude',
                    'delivery_latitude',
                    'delivery_longitude',
                    'order_date',
                    "order_time_hour",
                    "order_day",
                    "city_name",
                    "order_day_of_week",
                    "order_month"]

df.drop(columns=columns_to_drop, inplace=True)

df

,age,ratings,weather_conditions,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,time_taken (min),is_weekend,pickup_time_minutes,order_time_of_day,distance
0,36.0,4.2,Fog,jam,2,snack,motorcycle,3.0,no,metropolitian,46,0,15.0,night,10.28
1,21.0,4.7,Stormy,high,1,meal,motorcycle,1.0,no,metropolitian,23,1,10.0,afternoon,6.24
2,23.0,4.7,Sandstorms,medium,1,drinks,scooter,1.0,no,metropolitian,21,1,10.0,evening,13.79
3,34.0,4.3,Sandstorms,low,0,buffet,motorcycle,0.0,no,metropolitian,20,1,10.0,morning,2.93
4,24.0,4.7,Fog,jam,1,snack,scooter,1.0,no,metropolitian,41,0,15.0,evening,19.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45488,30.0,4.8,Windy,high,1,meal,motorcycle,0.0,no,metropolitian,32,0,10.0,morning,1.49
45489,21.0,4.6,Windy,jam,0,buffet,motorcycle,1.0,no,metropolitian,36,0,15.0,evening,NaN
45490,30.0,4.9,Cloudy,low,1,drinks,scooter,0.0,no,metropolitian,16,0,NaN,night,4.66
45491,20.0,4.7,Cloudy,high,0,snack,motorcycle,1.0,no,metropolitian,26,1,5.0,afternoon,6.23


## Drop missing values

In [12]:
temp_df = df.copy().dropna()

In [13]:
# split into X and y

X = temp_df.drop(columns='time_taken (min)')
y = temp_df['time_taken (min)']

X

,age,ratings,weather_conditions,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,is_weekend,pickup_time_minutes,order_time_of_day,distance
0,36.0,4.2,Fog,jam,2,snack,motorcycle,3.0,no,metropolitian,0,15.0,night,10.28
1,21.0,4.7,Stormy,high,1,meal,motorcycle,1.0,no,metropolitian,1,10.0,afternoon,6.24
2,23.0,4.7,Sandstorms,medium,1,drinks,scooter,1.0,no,metropolitian,1,10.0,evening,13.79
3,34.0,4.3,Sandstorms,low,0,buffet,motorcycle,0.0,no,metropolitian,1,10.0,morning,2.93
4,24.0,4.7,Fog,jam,1,snack,scooter,1.0,no,metropolitian,0,15.0,evening,19.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45485,30.0,4.2,Cloudy,medium,1,snack,motorcycle,2.0,yes,metropolitian,0,15.0,evening,10.45
45487,35.0,4.2,Windy,jam,2,drinks,motorcycle,1.0,no,metropolitian,0,10.0,night,16.60
45488,30.0,4.8,Windy,high,1,meal,motorcycle,0.0,no,metropolitian,0,10.0,morning,1.49
45491,20.0,4.7,Cloudy,high,0,snack,motorcycle,1.0,no,metropolitian,1,5.0,afternoon,6.23


In [14]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
X_train.isna().sum()

,0
age,0
ratings,0
weather_conditions,0
traffic,0
vehicle_condition,0
type_of_order,0
type_of_vehicle,0
multiple_deliveries,0
festival,0
city_type,0


In [16]:
# transform target column

pt = PowerTransformer()

y_train_pt = pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt = pt.transform(y_test.values.reshape(-1,1))

## Pre Processing Pipeline

In [17]:
# Assingning the columns

num_cols = ['age', 'ratings', 'pickup_time_minutes', 'distance']

nominal_cat_cols = ["weather_conditions", "type_of_order",
                    "type_of_vehicle","festival",
                    "city_type",
                    "is_weekend",
                    "order_time_of_day"]

ordinal_cat_cols = ['traffic']


In [18]:
len(num_cols + nominal_cat_cols + ordinal_cat_cols)

12

In [19]:
for col in ordinal_cat_cols:
    print(col,X_train[col].unique())

traffic ['medium' 'high' 'jam' 'low']


In [20]:
# build a preprocessor

traffic_order = ["low","medium","high","jam"]


preprocessor = ColumnTransformer(transformers=[
    ("scale", MinMaxScaler(), num_cols),
    ("nominal_encode", OneHotEncoder(drop="first",handle_unknown="ignore",
                                     sparse_output=False), nominal_cat_cols),
    ("ordinal_encode", OrdinalEncoder(categories=[traffic_order],
                                      encoded_missing_value=-999,
                                      handle_unknown="use_encoded_value",
                                      unknown_value=-1), ordinal_cat_cols)
],remainder="passthrough",n_jobs=-1,verbose_feature_names_out=False)


preprocessor

ColumnTransformer(n_jobs=-1, remainder='passthrough',
                  transformers=[('scale', MinMaxScaler(),
                                 ['age', 'ratings', 'pickup_time_minutes',
                                  'distance']),
                                ('nominal_encode',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather_conditions', 'type_of_order',
                                  'type_of_vehicle', 'festival', 'city_type',
                                  'is_weekend', 'order_time_of_day']),
                                ('ordinal_encode',
                                 OrdinalEncoder(categories=[['low', 'medium',
                                                             'high', 'jam']],
                                                encoded_missing_value=-999,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['traffic'])],
                  verbose_feature_names_out=False)

In [21]:
# build the pipeline

processing_pipeline = Pipeline(steps=[
                                # ("simple_imputer",simple_imputer),
                                ("preprocess",preprocessor)
                                # ("knn_imputer",knn_imputer)
                            ])

processing_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  ['age', 'ratings',
                                                   'pickup_time_minutes',
                                                   'distance']),
                                                 ('nominal_encode',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['weather_conditions',
                                                   'type_of_order',
                                                   'type_of_vehicle',
                                                   'festival', 'city_type',
                                                   'is_weekend',
                                                   'order_time_of_day']),
                                                 ('ordinal_encode',
                                                  OrdinalEncoder(categories=[['low',
                                                                              'medium',
                                                                              'high',
                                                                              'jam']],
                                                                 encoded_missing_value=-999,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['traffic'])],
                                   verbose_feature_names_out=False))])

In [22]:
# do data preprocessing

X_train_trans = processing_pipeline.fit_transform(X_train)

X_test_trans = processing_pipeline.transform(X_test)

In [23]:
X_train_trans

array([[0.31578947, 0.84      , 0.5       , ..., 1.        , 1.        ,
        1.        ],
       [0.94736842, 0.8       , 0.5       , ..., 2.        , 2.        ,
        0.        ],
       [0.26315789, 0.84      , 1.        , ..., 3.        , 0.        ,
        2.        ],
       ...,
       [0.68421053, 0.6       , 0.5       , ..., 3.        , 2.        ,
        1.        ],
       [0.73684211, 0.96      , 1.        , ..., 2.        , 1.        ,
        1.        ],
       [0.52631579, 0.72      , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [24]:
def objective(trial):
    with mlflow.start_run(nested=True):
        model_name = trial.suggest_categorical("model",["SVM","RF","KNN","GB","XGB","LGBM"])

        if model_name == "SVM":
            kernel_svm = trial.suggest_categorical("kernel_svm",["linear","poly","rbf"])
            if kernel_svm == "linear":
                c_linear = trial.suggest_float("c_linear",0,10)
                model = SVR(C=c_linear,kernel="linear")

            elif kernel_svm == "poly":
                c_poly = trial.suggest_float("c_poly",0,10)
                degree_poly = trial.suggest_int("degree_poly",1,5)
                model = SVR(C=c_poly,degree=degree_poly,
                            kernel="poly")

            else:
                c_rbf = trial.suggest_float("c_rbf",0,100)
                gamma_rbf = trial.suggest_float("gamma_rbf",0,10)
                model = SVR(C=c_rbf,gamma=gamma_rbf,
                            kernel="rbf")

        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,200)
            max_depth_rf = trial.suggest_int("max_depth_rf",2,20)
            model = RandomForestRegressor(n_estimators=n_estimators_rf,
                                        max_depth=max_depth_rf,
                                        random_state=42,
                                        n_jobs=-1)

        elif model_name == "GB":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,200)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",0,1)
            max_depth_gb = trial.suggest_int("max_depth_gb",2,20)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb,
                                                learning_rate=learning_rate_gb,
                                                max_depth=max_depth_gb,
                                                random_state=42)

        elif model_name == "KNN":
            n_neighbors_knn = trial.suggest_int("n_neighbors_knn",1,25)
            weights_knn = trial.suggest_categorical("weights_knn",["uniform","distance"])
            model = KNeighborsRegressor(n_neighbors=n_neighbors_knn,
                                        weights=weights_knn,n_jobs=-1)

        elif model_name == "XGB":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,200)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",0.1,0.5)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",2,20)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                    learning_rate=learning_rate_xgb,
                                    max_depth=max_depth_xgb,
                                    random_state=42,
                                    n_jobs=-1)

        elif model_name == "LGBM":
            n_estimators_lgbm = trial.suggest_int("n_estimators_lgbm",10,200)
            learning_rate_lgbm = trial.suggest_float("learning_rate_lgbm",0.1,0.5)
            max_depth_lgbm = trial.suggest_int("max_depth_lgbm",2,20)
            model = LGBMRegressor(n_estimators=n_estimators_lgbm,
                                    learning_rate=learning_rate_lgbm,
                                    max_depth=max_depth_lgbm,
                                    random_state=42)


        # train the model
        model.fit(X_train_trans,y_train_pt.ravel())

        # log model params
        mlflow.log_params(model.get_params())

        # get the predictions
        y_pred_train = model.predict(X_train_trans)
        y_pred_test = model.predict(X_test_trans)

        # get the actual predictions values
        y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
        y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

        # calculate the error
        error = mean_absolute_error(y_test,y_pred_test_org)

        # log model_name
        mlflow.log_param("model",model_name)

        # log error
        mlflow.log_metric("MAE",error)

        return error

In [25]:
# create optuna study
study = optuna.create_study(direction="minimize",study_name="model_selection")

with mlflow.start_run(run_name="Best Model") as parent:
    # optimize the objective function
    study.optimize(objective,n_trials=30,n_jobs=-1)

    # log the best parameters
    mlflow.log_params(study.best_params)

    # log the best score
    mlflow.log_metric("best_score",study.best_value)

[I 2025-02-08 15:54:42,766] A new study created in memory with name: model_selection
[I 2025-02-08 15:54:57,636] Trial 0 finished with value: 3.135304302070662 and parameters: {'model': 'RF', 'n_estimators_rf': 97, 'max_depth_rf': 20}. Best is trial 0 with value: 3.135304302070662.


🏃 View run unique-flea-592 at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2/runs/756b4592c4ca4c43b55b73484adabe3a
🧪 View experiment at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2


[I 2025-02-08 15:55:08,850] Trial 2 finished with value: 3.839330655978426 and parameters: {'model': 'GB', 'n_estimators_gb': 54, 'learning_rate_gb': 0.7508864839036913, 'max_depth_gb': 20}. Best is trial 0 with value: 3.135304302070662.


🏃 View run intelligent-boar-22 at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2/runs/64040e1efa9e481c8fedca733a3789f7
🧪 View experiment at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-02-08 15:55:09,996] Trial 3 finished with value: 3.246797197419913 and parameters: {'model': 'LGBM', 'n_estimators_lgbm': 29, 'learning_rate_lgbm': 0.2251597398421344, 'max_depth_lgbm': 5}. Best is trial 0 with value: 3.135304302070662.


...

🏃 View run charming-bug-340 at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2/runs/00e93037894240d1855bff8a29f81e22
🧪 View experiment at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2


[I 2025-02-08 16:08:54,813] Trial 29 finished with value: 4.451651970269369 and parameters: {'model': 'GB', 'n_estimators_gb': 160, 'learning_rate_gb': 0.013312219763460642, 'max_depth_gb': 3}. Best is trial 20 with value: 3.10541546339861.


🏃 View run monumental-hawk-277 at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2/runs/089abccd032248a6b237b326ff0756a4
🧪 View experiment at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2


[I 2025-02-08 16:19:52,504] Trial 8 finished with value: 3.987048369308125 and parameters: {'model': 'SVM', 'kernel_svm': 'poly', 'c_poly': 2.935322637787019, 'degree_poly': 5}. Best is trial 20 with value: 3.10541546339861.


🏃 View run popular-boar-123 at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2/runs/91424f7940754940be96fed9253558c5
🧪 View experiment at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2
🏃 View run Best Model at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2/runs/be13bb0786df4cdcb1310ed1e70499c0
🧪 View experiment at: https://dagshub.com/mukeshjangid7877/zomato-delivery-time-prediction.mlflow/#/experiments/2


In [26]:
# best score

study.best_value

3.10541546339861

In [27]:
lgbm_params = {
    "n_estimators": 145,
    "learning_rate": 0.16632111599858262,
    "max_depth": 17
}

In [28]:
# train the model on best parameters

lgbm = LGBMRegressor(**lgbm_params)

lgbm.fit(X_train_trans,y_train_pt.ravel())

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002054 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 344
[LightGBM] [Info] Number of data points in the train set: 24604, number of used features: 24
[LightGBM] [Info] Start training from score -0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LGBMRegressor(learning_rate=0.16632111599858262, max_depth=17, n_estimators=145)

In [29]:
# get the predictions
y_pred_train = lgbm.predict(X_train_trans)
y_pred_test = lgbm.predict(X_test_trans)

# get the actual predictions values

y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [30]:

print(f"The train error is {mean_absolute_error(y_train,y_pred_train_org):.2f} minutes")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test_org):.2f} minutes")

The train error is 2.76 minutes
The test error is 3.09 minutes


In [31]:
print(f"The train r2 score is {r2_score(y_train,y_pred_train_org):.2f}")
print(f"The test r2 score is {r2_score(y_test,y_pred_test_org):.2f}")

The train r2 score is 0.86
The test r2 score is 0.84


In [32]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_c_linear,params_c_poly,params_c_rbf,params_degree_poly,params_gamma_rbf,...,params_max_depth_rf,params_max_depth_xgb,params_model,params_n_estimators_gb,params_n_estimators_lgbm,params_n_estimators_rf,params_n_estimators_xgb,params_n_neighbors_knn,params_weights_knn,state
0,0,3.135304,2025-02-08 15:54:43.101196,2025-02-08 15:54:57.636617,0 days 00:00:14.535421,NaN,NaN,NaN,NaN,NaN,...,20.0,NaN,RF,NaN,NaN,97.0,NaN,NaN,NaN,COMPLETE
1,1,3.621990,2025-02-08 15:54:43.102506,2025-02-08 15:55:42.203103,0 days 00:00:59.100597,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GB,191.0,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,3.839331,2025-02-08 15:54:57.641721,2025-02-08 15:55:08.849866,0 days 00:00:11.208145,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GB,54.0,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,3.246797,2025-02-08 15:55:08.855580,2025-02-08 15:55:09.996449,0 days 00:00:01.140869,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LGBM,NaN,29.0,NaN,NaN,NaN,NaN,COMPLETE
4,4,3.123998,2025-02-08 15:55:10.002980,2025-02-08 15:55:28.607951,0 days 00:00:18.604971,NaN,NaN,NaN,NaN,NaN,...,17.0,NaN,RF,NaN,NaN,149.0,NaN,NaN,NaN,COMPLETE
5,5,3.211858,2025-02-08 15:55:28.612792,2025-02-08 15:55:29.989421,0 days 00:00:01.376629,NaN,NaN,NaN,NaN,NaN,...,NaN,6.0,XGB,NaN,NaN,NaN,13.0,NaN,NaN,COMPLETE
6,6,5.754674,2025-02-08 15:55:29.994271,2025-02-08 15:55:39.285637,0 days 00:00:09.291366,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,KNN,NaN,NaN,NaN,NaN,1.0,uniform,COMPLETE
7,7,3.166773,2025-02-08 15:55:39.291282,2025-02-08 15:55:45.172939,0 days 00:00:05.881657,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LGBM,NaN,188.0,NaN,NaN,NaN,NaN,COMPLETE
8,8,3.987048,2025-02-08 15:55:42.206698,2025-02-08 16:19:52.504594,0 days 00:24:10.297896,NaN,2.935323,NaN,5.0,NaN,...,NaN,NaN,SVM,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETE
9,9,3.467185,2025-02-08 15:55:45.181603,2025-02-08 15:55:56.262372,0 days 00:00:11.080769,NaN,NaN,NaN,NaN,NaN,...,NaN,20.0,XGB,NaN,NaN,NaN,37.0,NaN,NaN,COMPLETE


In [35]:
# model frequency

study.trials_dataframe()['params_model'].value_counts()

,count
params_model,
RF,14
SVM,4
GB,3
LGBM,3
XGB,3
KNN,3


In [34]:
# avg scores for all tested models

study.trials_dataframe().groupby("params_model")['value'].mean().sort_values()

,value
params_model,
LGBM,3.212735
RF,3.233043
XGB,3.471864
GB,3.970991
SVM,4.860426
KNN,5.093004


In [37]:
from sklearn.compose import TransformedTargetRegressor


model = TransformedTargetRegressor(regressor=lgbm,
                                    transformer=pt)

In [38]:

scores = cross_val_score(model,
                         X_train_trans,
                         y_train,
                         scoring="neg_mean_absolute_error",
                         cv=5,n_jobs=-1)

scores

array([-3.0728815 , -3.08517133, -3.04127003, -3.0877709 , -3.08533037])

In [39]:
# mean score

- scores.mean()

3.074484827222065

In [40]:
optuna.visualization.plot_optimization_history(study)
